In [26]:
# SETUP
using JuMP
using GLPK
using Random
using LinearAlgebra
using Distributions
include("dynamic_functions.jl")

solve_model (generic function with 3 methods)

# no retention yet 
we start with a matching model with learning:
- workers have noisy signal 
- firm remember their true abilities once employed
- firms and workers rematch after each period 

Task: we will graph the total surplus achieved after N periods of switching
ideally we should see the surplus increases as firm learn about the workers 

In [ ]:
# set up 
no_w, no_f = 5,5
no_period = 10 #number of period
noisy_level = 2
ability = rand(Normal(0,1), no_w,no_f)

signal = ability .+ rand(Normal(0, noisy_level), size(ability))
# display(ability)

for i in 1:no_period
    # display(signal)
    match = solve_model(signal)
    # display(match)
    for i in 1:no_w, j in 1:no_f
        if match[i,j] == 1
            signal[i,j] = ability[i,j] #signal becomes true ablity as firms learn
        end
    end
    println(sum(ability .* match))
end

opt_match = solve_model(ability)
print("optimal match would yield: ", sum(ability .* opt_match))


0.1591213359461363
3.6327848078038363
2.1161333159328017
2.2184369584388994
3.895292471860761
3.895292471860761
3.895292471860761
3.895292471860761
3.895292471860761
3.895292471860761
optimal match would yield: 4.409740901151579

# with retention where firm can fire workers 